In [2]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])


train_data = datasets.CIFAR10('data',train=True, download = True,transform=transform)
test_data = datasets.CIFAR10('data',train=False, download = True,transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
print("no of training samples : ",len(train_data))
print("no of test samples : ",len(test_data))
print("Shape : ",train_data[0][0])

no of training samples :  50000
no of test samples :  10000
Shape :  tensor([[[-0.5373, -0.6627, -0.6078,  ...,  0.2392,  0.1922,  0.1608],
         [-0.8745, -1.0000, -0.8588,  ..., -0.0353, -0.0667, -0.0431],
         [-0.8039, -0.8745, -0.6157,  ..., -0.0745, -0.0588, -0.1451],
         ...,
         [ 0.6314,  0.5765,  0.5529,  ...,  0.2549, -0.5608, -0.5843],
         [ 0.4118,  0.3569,  0.4588,  ...,  0.4431, -0.2392, -0.3490],
         [ 0.3882,  0.3176,  0.4039,  ...,  0.6941,  0.1843, -0.0353]],

        [[-0.5137, -0.6392, -0.6235,  ...,  0.0353, -0.0196, -0.0275],
         [-0.8431, -1.0000, -0.9373,  ..., -0.3098, -0.3490, -0.3176],
         [-0.8118, -0.9451, -0.7882,  ..., -0.3412, -0.3412, -0.4275],
         ...,
         [ 0.3333,  0.2000,  0.2627,  ...,  0.0431, -0.7569, -0.7333],
         [ 0.0902, -0.0353,  0.1294,  ...,  0.1608, -0.5137, -0.5843],
         [ 0.1294,  0.0118,  0.1137,  ...,  0.4431, -0.0745, -0.2784]],

        [[-0.5059, -0.6471, -0.6627,  ..., -0.1

In [5]:
train_set  = torch.utils.data.DataLoader(train_data,batch_size=32)
test_set  = torch.utils.data.DataLoader(test_data,batch_size=32)

In [6]:
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck']

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
    
    def __init__(self):
        super(Classifier,self).__init__()
        
        self.conv1 = nn.Conv2d(3,16,3,padding=1)
        self.conv2 = nn.Conv2d(16,32,3,padding=1)
        self.conv3 = nn.Conv2d(32,64,3,padding=1)
        
        self.maxpool = nn.MaxPool2d(2,2)
        
        self.fc1 = nn.Linear(4*4*64,500)
        self.fc2 = nn.Linear(500,10)
        
        
    def forward(self,x):
        
        x = self.maxpool(F.relu(self.conv1(x)))
        x = self.maxpool(F.relu(self.conv2(x)))
        x = self.maxpool(F.relu(self.conv3(x)))
        
        x = x.view(-1,4*4*64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
        
        


In [8]:
classifier = Classifier()

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(),lr=0.001)

In [19]:
def train(epochs, data,model):
    
    for epoch in range(epochs) :
        
        train_loss=0
        model.train()
        for x,y in data:
#             print("labels ",y)
#             print(x)
            
            if torch.cuda.is_available():
                x,y = x.cuda(),y.cuda()
            
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out,y)
            loss.backward()
            optimizer.step()
            
            train_loss+=loss.item()
            
        
        print("train loss epoch {} = {}".format(epoch+1,train_loss/len(data)))
            

In [20]:
train(2,train_set,classifier)

train loss epoch 1 = 0.6313993941875734
train loss epoch 2 = 0.528344972863536
